### THIS NOTEBOOK WAS USED FOR Chapter 4.2.4 third iteration 365 folds


In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
)

import matplotlib.pyplot as plt
from IPython.display import display
import shap

# ------------------------------------------------------------------
# 1. LOAD & PREPARE DATA 
# ------------------------------------------------------------------

# InputData CSV exported from Process mining platform 
df = pd.read_csv("InputData3.2.4.csv")
data = df.copy()

# Filtering based on cutoff date and changing the datatypes to date columns
cutoff_dateclose = pd.Timestamp("2025-1-12")
cutoff_datecreate = pd.Timestamp("2023-1-1")

data["CloseDateNew"] = pd.to_datetime(data["CloseDateNew"])
data["CreatedDate"] = pd.to_datetime(data["CreatedDate"])
data["Timestamp"] = pd.to_datetime(data["Timestamp"])

# Filter by segment if needed
# data = data[data["MarketSegmentc"] == "NAM"]

# Choose your cutoff logic:
# data = data[data["CloseDateNew"] > cutoff_dateclose]
# data = data[(data["CreatedDate"] > cutoff_datecreate ) & (data["CloseDateNew"] > cutoff_dateclose)]
data = data[data["CreatedDate"] > cutoff_datecreate]  # current choice
data = data[data["Timestamp"] > cutoff_datecreate]    # current choice

# Only closed opportunities (comment out to include open ones)
data = data[data["IsClosed"] == 1]

# Values to remove from Activity as these might be leaky
to_drop = ["CloseWon", "ClosedLost", "DidNotMaterialise", "ChangeCloseDate"]
data = data[~data["Activity"].isin(to_drop)].copy()

print(data["Activity"].value_counts())
print(data.shape)
data.info()

n_unique_opps = df["SFOpportunity ID"].nunique()
print("Number of distinct opportunities:", n_unique_opps)

# ------------------------------------------------------------------
# 2. HANDLE MISSING & ENCODE
# ------------------------------------------------------------------

# Columns to explicitly drop (helper columns not used in training)
explicit_drop = [
    "Customer",
    "SFOpportunity ID",
    "IsClosed",
    "CloseDateNew",
    "CreatedDate",
]

# Find columns with >60% missing values
missing_pct = data.isnull().sum() / len(data) * 100
high_missing = missing_pct[missing_pct > 60].index.tolist()

# Combine all columns to drop
cols_to_drop = list(set(explicit_drop + high_missing))

data_cleaned = data.drop(columns=cols_to_drop)

# Missing values summary
missing_counts = data_cleaned.isnull().sum()
missing_pct = (missing_counts / len(data_cleaned) * 100).round(2)

missing_summary = pd.DataFrame({
    "Missing Values": missing_counts,
    "Missing %": missing_pct
})

display(missing_summary)

# Fill missing in categorical columns
for col in data_cleaned.select_dtypes(include="object").columns:
    data_cleaned[col] = data_cleaned[col].fillna("Unknown")

# Fill missing in numeric columns
for col in data_cleaned.select_dtypes(include=["float64", "int64"]).columns:
    if col == "CustomerPrevRevenue":
        # Special rule: treat missing as 0 (no previous revenue)
        data_cleaned[col] = data_cleaned[col].fillna(0)
    else:
        if data_cleaned[col].isnull().any():
            median_val = data_cleaned[col].median()
            data_cleaned[col] = data_cleaned[col].fillna(median_val)

# Data summary
summary = pd.DataFrame({
    "Unique Values": data_cleaned.nunique(dropna=True),
    "Missing Values": data_cleaned.isnull().sum(),
    "Missing %": (data_cleaned.isnull().sum() / len(data_cleaned) * 100).round(2),
    "Data Type": data_cleaned.dtypes.astype(str)
}).sort_values("Unique Values", ascending=False)

display(summary)

# One-hot encode categoricals
df_encoded = pd.get_dummies(data_cleaned, drop_first=False)

# ------------------------------------------------------------------
# 3. TIME-SERIES SPLIT SETUP (365 folds)
# ------------------------------------------------------------------

# Ensure Timestamp is datetime and sort by time
df_encoded["Timestamp"] = pd.to_datetime(df_encoded["Timestamp"])
df_encoded = df_encoded.sort_values("Timestamp").reset_index(drop=True)

# Features and target
X = df_encoded.drop(columns=["IsWon", "Timestamp"])
y = df_encoded["IsWon"]

# Time series split (365 folds)
# NOTE: This requires enough rows; if you get errors,
# reduce n_splits or increase data size.
tscv = TimeSeriesSplit(n_splits=365)

fold_rows = []   # store per-fold date ranges

for fold, (train_idx, test_idx) in enumerate(tscv.split(X, y), start=1):
    train_start = df_encoded.loc[train_idx[0], "Timestamp"]
    train_end   = df_encoded.loc[train_idx[-1], "Timestamp"]
    test_start  = df_encoded.loc[test_idx[0], "Timestamp"]
    test_end    = df_encoded.loc[test_idx[-1], "Timestamp"]

    print(f"Fold {fold}:")
    print("  Train period:", train_start, "→", train_end)
    print("  Test  period:", test_start,  "→", test_end)
    print("  Train shape:", len(train_idx), "rows, Test shape:", len(test_idx), "rows")

    fold_rows.append({
        "fold": fold,
        "train_start": train_start,
        "train_end": train_end,
        "test_start": test_start,
        "test_end": test_end,
    })

# ------------------------------------------------------------------
# 4. MODELING - ONLY XGBOOST 
# ------------------------------------------------------------------

# Helper function for evaluation
def evaluate_model(name, y_true, y_pred, y_proba):
    return {
        "Model": name,
        "ROC_AUC": roc_auc_score(y_true, y_proba),
        "PR_AUC": average_precision_score(y_true, y_proba),
        "Accuracy": accuracy_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "F1": f1_score(y_true, y_pred),
    }

results = []

# Keep references to last-fold XGB for SHAP
last_fold_xgb = None
last_fold_X_test = None
last_fold_y_test = None
last_fold_y_pred = None
last_fold_y_proba = None

for fold, (train_idx, test_idx) in enumerate(tscv.split(X, y), start=1):
    print(f"\n=== Fold {fold} ===")
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    test_win_rate = (y_test.mean() * 100).round(2)  # proportion of 1s

    # ---------------------------------------------------
    # XGBoost
    # ---------------------------------------------------
    xgb_clf = xgb.XGBClassifier(
        eval_metric="logloss",
        random_state=42
    )
    xgb_clf.fit(X_train, y_train)

    y_pred_xgb = xgb_clf.predict(X_test)
    y_proba_xgb = xgb_clf.predict_proba(X_test)[:, 1]

    res_xgb = evaluate_model("XGBoost", y_test, y_pred_xgb, y_proba_xgb)
    res_xgb["fold"] = fold
    res_xgb["test_win_rate"] = test_win_rate
    results.append(res_xgb)

    print("  Test win rate (% of 1s):", test_win_rate)
    print("  XGBoost ROC-AUC:", res_xgb["ROC_AUC"])
    print("  XGBoost PR-AUC:", res_xgb["PR_AUC"])
    print("  XGBoost Accuracy:", res_xgb["Accuracy"])

    # Keep last fold XGBoost for SHAP
    last_fold_xgb = xgb_clf
    last_fold_X_test = X_test.copy()
    last_fold_y_test = y_test.copy()
    last_fold_y_pred = y_pred_xgb.copy()
    last_fold_y_proba = y_proba_xgb.copy()

# ------------------------------------------------------------------
# 5. RESULTS TABLES & EXPORT
# ------------------------------------------------------------------

results_df = pd.DataFrame(results)

# Per fold (only XGBoost)
by_fold = (
    results_df
    .groupby(["fold", "Model"], as_index=False)[
        ["ROC_AUC", "PR_AUC", "Accuracy", "Recall", "Precision", "F1", "test_win_rate"]
    ]
    .mean()
    .round(4)
    .sort_values(["fold", "Model"])
)

display(by_fold.head())

# Average over folds
avg_metrics = (
    results_df
    .groupby("Model", as_index=False)[
        ["ROC_AUC", "PR_AUC", "Accuracy", "Recall", "Precision", "F1"]
    ]
    .mean()
    .round(4)
    .sort_values("ROC_AUC", ascending=False)
)

print("\nAverage metrics over all folds:")
display(avg_metrics)

# Save detailed per-fold results
fold_periods_df = pd.DataFrame(fold_rows).sort_values("fold").reset_index(drop=True)

by_fold_with_dates = by_fold.merge(
    fold_periods_df,
    on="fold",
    how="left"
).sort_values(["Model", "fold"])

by_fold_with_dates.to_csv("by_fold_xgboost_365folds.csv", index=False)
results_df.to_csv("xgboost_evaluation_results_detailed.csv", index=False)

# Aggregate average and standard deviation of metrics
metric_cols = ["ROC_AUC", "PR_AUC", "Accuracy", "Recall", "Precision", "F1", "test_win_rate"]

summary = (
    results_df
    .groupby("Model")[metric_cols]
    .agg(["mean", "std"])
)

summary.columns = [f"{metric}_{stat}" for metric, stat in summary.columns]
summary = summary.round(4).reset_index()

print("Average and standard deviation of metrics (XGBoost):")
print(summary.to_string(index=False))

summary.to_csv("xgboost_metrics_avg_std_365folds.csv", index=False)

# ------------------------------------------------------------------
# 6. PLOT METRICS PER FOLD (XGBOOST ONLY)
# ------------------------------------------------------------------

metrics = ["ROC_AUC", "PR_AUC", "Accuracy", "Precision", "Recall", "F1"]

fig, axes = plt.subplots(len(metrics), 1, figsize=(10, 14), sharex=True)

for ax, metric in zip(axes, metrics):
    subset = by_fold[by_fold["Model"] == "XGBoost"]
    ax.plot(subset["fold"], subset[metric], marker="o", label="XGBoost")
    ax.set_ylabel(metric)
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel("Fold")
axes[0].legend(loc="best")
plt.tight_layout()
plt.show()

# ------------------------------------------------------------------
# 7. SHAP ANALYSIS FOR LAST FOLD XGBOOST
# ------------------------------------------------------------------

if last_fold_xgb is not None and last_fold_X_test is not None:
    print("\nRunning SHAP for last fold XGBoost model...")
    explainer = shap.TreeExplainer(last_fold_xgb)
    shap_values = explainer.shap_values(last_fold_X_test)

    # Bar summary (global feature importance)
    shap.summary_plot(shap_values, last_fold_X_test, plot_type="bar")

    # Beeswarm detailed summary
    shap.summary_plot(shap_values, last_fold_X_test)

Series([], Name: count, dtype: int64)
(0, 26)
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   SFOpportunity ID                  0 non-null      int64         
 1   IsWon                             0 non-null      int64         
 2   IsClosed                          0 non-null      int64         
 3   Activity                          0 non-null      float64       
 4   Timestamp                         0 non-null      datetime64[ns]
 5   CloseDateNew                      0 non-null      datetime64[ns]
 6   CreatedDate                       0 non-null      datetime64[ns]
 7   Previous activity                 0 non-null      float64       
 8   Lead time from previous activity  0 non-null      float64       
 9   Activity amount so far            0 non-null      float64       
 10  Rework amount        

,Missing Values,Missing %
IsWon,0,NaN
Activity,0,NaN
Timestamp,0,NaN
Previous activity,0,NaN
Lead time from previous activity,0,NaN
Activity amount so far,0,NaN
Rework amount,0,NaN
Lead time until the stage,0,NaN
Amount in stage,0,NaN
MarketSegmentc,0,NaN


,Unique Values,Missing Values,Missing %,Data Type
IsWon,0,0,NaN,int64
Industryc,0,0,NaN,float64
CustomerPrevRevenue,0,0,NaN,float64
Tenderflag,0,0,NaN,float64
BigDealAlertSent,0,0,NaN,float64
OfferingCategory,0,0,NaN,float64
QuoteOperatingUnitc,0,0,NaN,float64
Paymentterm,0,0,NaN,float64
Ownerrolec,0,0,NaN,float64
Subsegmentc,0,0,NaN,float64


ValueError: Cannot have number of folds=366 greater than the number of samples=0.

### VISUALIZE THE CONFUSION MATRIX FROM THE LAST FOLD

In [4]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# ------------------------------------------------------------------
# CONFUSION MATRIX FOR LAST FOLD (MOST RECENT PERIOD)
# ------------------------------------------------------------------

if last_fold_y_test is not None and last_fold_y_pred is not None:
    # Raw counts
    cm = confusion_matrix(last_fold_y_test, last_fold_y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
    disp.plot(values_format='d')
    plt.title("Confusion Matrix - Last Fold (Most Recent Period)")
    plt.show()



In [9]:
display(by_fold.tail())


,fold,Model,ROC_AUC,PR_AUC,Accuracy,Recall,Precision,F1,test_win_rate
360,361,XGBoost,0.7549,0.9553,0.8758,0.9667,0.9000,0.9321,88.24
361,362,XGBoost,0.7368,0.9661,0.8987,0.9891,0.9064,0.9459,89.54
362,363,XGBoost,0.8302,0.9813,0.9085,0.9472,0.9539,0.9505,92.81
363,364,XGBoost,0.8447,0.9905,0.9641,0.9897,0.9731,0.9813,95.42
364,365,XGBoost,0.8760,0.9859,0.9477,0.9894,0.9555,0.9721,92.16


## VISUALIZE THE TIMESPLIT

C:\Users\OSNIE\AppData\Local\Temp\ipykernel_16832\3615464373.py:75: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from sklearn.model_selection import TimeSeriesSplit

# -------------------------------------------------------------
# TimeSeriesSplit visualization (time range on x-axis)
# -------------------------------------------------------------

n_splits = 20                     # same as in your training
tscv = TimeSeriesSplit(n_splits=n_splits)
splits = list(tscv.split(X, y))   # use same logic as for training

n_samples = len(X)
indices = np.arange(n_samples)
cmap_cv = plt.cm.coolwarm

fig, ax = plt.subplots(figsize=(10, 6))

# Plot train / test indices per fold
for idx, (train_idx, test_idx) in enumerate(splits):
    # initialize all as NaN (not drawn)
    fold_colors = np.full(n_samples, np.nan)
    fold_colors[train_idx] = 0   # train
    fold_colors[test_idx] = 1    # test

    ax.scatter(
        indices,
        np.full_like(indices, idx + 0.5),  # vertical position for this fold
        c=fold_colors,
        marker="s",
        s=10,
        cmap=cmap_cv,
        vmin=-0.2,
        vmax=1.2,
    )

# Optional: line of all samples at bottom
ax.scatter(
    indices,
    np.full_like(indices, -0.5),
    c="k",
    marker="|",
    s=50,
    label="Samples",
)

# Use first & last timestamps as x-axis labels
start_time = df_encoded["Timestamp"].iloc[0]
end_time   = df_encoded["Timestamp"].iloc[-1]

ax.set_ylabel("Fold")
ax.set_yticks(np.arange(n_splits) + 0.5)
ax.set_yticklabels([f"Fold {i+1}" for i in range(n_splits)])
ax.set_ylim(-1, n_splits + 1)
ax.set_xlim(0, n_samples)

ax.set_xlabel("Time (from first to last Timestamp)")
ax.set_xticks([0, n_samples - 1])
ax.set_xticklabels([
    start_time.strftime("%Y-%m-%d"),
    end_time.strftime("%Y-%m-%d"),
])

ax.set_title("TimeSeriesSplit folds (train/test indices)")

# Legend
legend_elements = [
    Patch(facecolor=cmap_cv(0.0), edgecolor="k", label="Train"),
    Patch(facecolor=cmap_cv(1.0), edgecolor="k", label="Test"),
]
ax.legend(handles=legend_elements, loc="upper right")
fig.tight_layout()
fig.savefig("timeseries_folds2.jpg", format="jpg", dpi=300, bbox_inches="tight")
plt.tight_layout()
plt.show()